In [1]:
#Importing importing Files

import re
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_lg")

In [2]:
#Reading File Data

fileData=[]
with open("Output.txt", "r") as file:
    for line in file:
        x = re.sub("=", " = ", line)
        x = re.sub("  "," ", x)   
        fileData.append(x)

In [3]:
#Loading Shelf Data from "Data Generator"
import shelve
import os

dest_file="NLP_Shelfv8//" + "Dict_Lists" + ".shlf"

shelf = shelve.open(dest_file)

dataType_number = shelf["dataType_number"]
dataType_char = shelf["dataType_char"]
dataType_list = shelf["dataType_list"]
dataType_bool = shelf["dataType_bool"]
dataType_class = shelf["dataType_class"]
dataType_void = shelf["dataType_void"]
dataType = shelf["dataType"]

variables_list = shelf["variables_list"]

scope_of_variable = shelf["scope_of_variable"]

operators = shelf["operators"]

command_declare = shelf["command_declare"]
command_print = shelf["command_print"]
command_input = shelf["command_input"]

func_prog = shelf["func_prog"]

special_class = shelf["special_class"]
special_function = shelf["special_function"]

shelf.close()

#print(dataType_number, "\n")
#print(dataType, "\n")
#print(variables_list, "\n")
#print(scope_of_variable, "\n")
#print(operators, "\n")
#print(command_declare, "\n")
#print(command_print, "\n")
#print(command_input, "\n")
#print(func_prog, "\n")
#print(special_class, "\n")
#print(special_function, "\n")

In [4]:
#Defining Patterns
#{"IN":["","s"]}
matcher = Matcher(nlp.vocab)

pattern1_1 = [{"LOWER":{"IN":dataType}}]
matcher.add("DATATYPE", None, pattern1_1)

pattern2_1 = [{"LOWER":"global"}]
pattern2_2 = [{"LOWER":"local"}]
matcher.add("SCOPE", None, pattern2_1, pattern2_2)

pattern3_1 = [{"LOWER":{"IN":operators}}]
pattern3_2 = [{"LOWER":{"IN":["additions", "adds"]}}]
pattern3_3 = [{"LOWER":{"IN":["subtractions", "subs"]}}]
pattern3_4 = [{"LOWER":{"IN":["minuses", "multiplies", "muls"]}}]
pattern3_5 = [{"LOWER":{"IN":["divides", "divs"]}}]
pattern3_6 = [{"LOWER":{"IN":["modulus", "moduluses", "mods"]}}]
matcher.add("OPERATOR", None, pattern3_1, pattern3_2, pattern3_3, pattern3_4, pattern3_5, pattern3_6)

pattern4_1 = [{"LOWER":{"IN":command_declare}}]
pattern4_2 = [{"LOWER":{"IN":command_print}}]
pattern4_3 = [{"LOWER":{"IN":command_input}}]
pattern4_4 = [{"LOWER":{"IN":special_function}}]
matcher.add("COMMAND", None, pattern4_1, pattern4_2, pattern4_3, pattern4_4)

pattern5_1 = [{"LOWER":{"IN":["function","functions"]}}]
pattern5_2 = [{"LOWER":{"IN":["func","funcs"]}}]
pattern5_3 = [{"LOWER":{"IN":["program","programs"]}}]
pattern5_4 = [{"LOWER":{"IN":["programme","programmes"]}}]
pattern5_5 = [{"LOWER":{"IN":["prog","progs"]}}]

pattern5_1 = [{"LOWER":{"IN":func_prog}}]
pattern5_2 = [{"LOWER":{"IN":["funcs","functions"]}}]
pattern5_3 = [{"LOWER":{"IN":["programs","programmes","progs"]}}]
matcher.add("FUNC_PROG", None, pattern5_1, pattern5_2, pattern5_3)

pattern6_1 = [{"LOWER": {"IN": variables_list}}]
matcher.add("VARI", None, pattern6_1)

pattern7_1 = [{"LOWER": {"IN": special_class}}]
matcher.add("SPL_CLASS", None, pattern7_1)

pattern8_1 = [{"LIKE_NUM": True}]
matcher.add("VALUE", None, pattern8_1)

In [5]:
#Creating Training Data

TRAINING_DATA = []

# Create a Doc object for each text in TEXTS
for doc in nlp.pipe(fileData):
    
    #x = re.sub("=", " = ", doc)
    #doc = re.sub("  "," ", x)
    
    # Match on the doc and create a list of matched spans
    spans = [doc[start:end] for match_id, start, end in matcher(doc)]
    
    #Modified Start
    entity_label=[]
    entity_label = [(nlp.vocab.strings[match_id]) for match_id, start, end in matcher(doc)]
    #if entity_label!=[]:
    #    print (entity_label)
    entities = [(span.start_char, span.end_char, entity_label[index]) for index,span in enumerate(spans)]
    
    #for eachWord in entities:
    #    print("Entities: {}".format(eachWord))
    
    # Format the matches as a (doc.text, entities) tuple
    training_example = (doc.text, {"entities": entities})
    
    # Append the example to the training data
    TRAINING_DATA.append(training_example)
    #Modified End
    
    '''
    entities = [(span.start_char, span.end_char, "INITIALIZATION") for span in spans]
    
    for eachWord in entities:
        print("Entities: {}".format(eachWord))
        
    # Format the matches as a (doc.text, entities) tuple
    training_example = (doc.text, {"entities": entities})
    # Append the example to the training data
    TRAINING_DATA.append(training_example)
    '''
    

#print(*TRAINING_DATA, sep="\n")


In [6]:
#Store to Data file for inspection
with open("training_data.txt","w") as o_file:
    for item in TRAINING_DATA:
        o_file.write("{}\n".format(item))

In [7]:
#Training Loop
import random

nlp = spacy.blank("en")
ner = nlp.create_pipe("ner")
nlp.add_pipe(ner)

ner.add_label("DATATYPE")
ner.add_label("SCOPE")
ner.add_label("OPERATOR")
ner.add_label("COMMAND")
ner.add_label("FUNC_PROG")
ner.add_label("VARI")
ner.add_label("SPL_CLASS")
ner.add_label("VALUE")

# Start the training
nlp.begin_training()

# Loop for 10 iterations
for itn in range(3):
    # Shuffle the training data
    random.shuffle(TRAINING_DATA)
    losses = {}

    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(TRAINING_DATA, size=1000):
        texts = [text for text, entities in batch]
        annotations = [entities for text, entities in batch]

        # Update the model
        nlp.update(texts, annotations, losses=losses)
    print(losses)

D:\Applications\miniconda3\envs\ME-NLP\lib\site-packages\spacy\language.py:635: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  proc.begin_training(
D:\Applications\miniconda3\envs\ME-NLP\lib\site-packages\spacy\language.py:635: UserWarning: [W034] Please install the package spacy-lookups-data in order to include the default lexeme normalization table for the language 'en'.
  proc.begin_training(


{'ner': 121742.77099967345}
{'ner': 0.04522313165110155}
{'ner': 9.772718850629942e-08}


In [8]:
from spacy import displacy

doc = nlp("Print variable i to log")
for ent in doc.ents:
    print (ent.text, ent.start_char, ent.end_char, ent.label_)

displacy.render(nlp(doc.text), style="ent", jupyter=True)

Print 0 5 COMMAND
variable 6 14 DATATYPE
i 15 16 VARI
log 20 23 SPL_CLASS


In [9]:
doc = nlp("set var num1 and v")
for ent in doc.ents:
    print (ent.text, ent.start_char, ent.end_char, ent.label_)

displacy.render(nlp(doc.text), style="ent", jupyter=True)

set 0 3 COMMAND
var 4 7 DATATYPE
num1 8 12 VARI
v 17 18 VARI


In [12]:
doc = nlp("create a function to add numbers z and b")
#doc = nlp("if a > b then add a and b")
#doc = nlp("get input k")
#doc = nlp("if a>b then print a. If b>c then print c")
for ent in doc.ents:
    print (ent.text, ent.start_char, ent.end_char, ent.label_)

displacy.render(nlp(doc.text), style="ent", jupyter=True)

create 0 6 COMMAND
function 9 17 FUNC_PROG
add 21 24 OPERATOR
numbers 25 32 DATATYPE
z 33 34 VARI
b 39 40 VARI


In [9]:
#Saving the nlp Object
nlp.to_disk("./NLP_Object")

In [ ]:
#Importing the nlp Object
import spacy
from spacy import displacy
nlp = spacy.load("./NLP_Object")

doc = nlp("create a function to add numbers a and b")
for ent in doc.ents:
    print (ent.text, ent.start_char, ent.end_char, ent.label_)

displacy.render(nlp(doc.text), style="ent", jupyter=True)